In [1]:
import sys

sys.path.append("..")

from src.ssunet.dataloader import BinomDataset, ValidationDataset
from src.ssunet import SSUnet, load_config

config = load_config("config/config.yml")
data = config.path_config.load_ssunet_data()
data.binxy(mode="max")

model = SSUnet(config.model_config)
train_data = BinomDataset(data, config.data_config, config.split_params)
validation_data = ValidationDataset
train_loader = config.loader_config.loader(train_data)
trainer = config.train_config.trainer
print(f"input_size: {tuple(next(iter(train_loader))[1].shape)}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


input_size: (12, 1, 32, 128, 128)


In [2]:
trainer.fit(model, train_loader, train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params | Mode 
--------------------------------------------------
0 | down_convs | ModuleList | 9.6 M  | train
1 | up_convs   | ModuleList | 5.1 M  | train
2 | conv_final | Sequential | 33     | train
--------------------------------------------------
14.7 M    Trainable params
0         Non-trainable params
14.7 M    Total params
58.718    Total estimated model params size (MB)
98        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]